# Sample Inflation
This competition evaluates only head of household prediction. In my previous trial, I found that prediction with non-head of household samples is much inferier than one without non-head of household. 

However, after dropping non-head of household samples, the dataset size decreases to one third. Sample size is a matter for the prediction accuracy. Then, I tried to leave some samples which have properties like head of household.

My procedure is follwing ...
1. Generate a model to calssify wheather a sample is head or non-head of household
2. Output probabilities that the sample is head of household
3. Leave high probability samples 



# result
* Socre was not improvement

In [ ]:
input_dir = '../input/'
working_dir = '../working/'

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
train = pd.read_csv(os.path.join(input_dir, 'train.csv'))
test = pd.read_csv(os.path.join(input_dir, 'test.csv'))

# Set index
train.index = train['Id'].values
test.index = test['Id'].values

# Pick Target
train_target = train['Target']

# Union train and test
all_data = pd.concat([train.drop('Target', axis=1), test], axis=0)

print(train.shape)
print(test.shape)
print(all_data.shape)

In [ ]:
# data cleaning
# copy from https://www.kaggle.com/katacs/data-cleaning-and-random-forest
# and make a change it
def data_cleaning(data):
    data['dependency']=np.sqrt(data['SQBdependency'])
    data['rez_esc']=data['rez_esc'].fillna(0)
    data['v18q1']=data['v18q1'].fillna(0)
    data['v2a1']=data['v2a1'].fillna(0)
    
    data['edjefa'] = data['edjefa'].replace({'no': 0, 'yes': 1})
    data['edjefa'] = data['edjefa'].astype('int')
    data['edjefe'] = data['edjefe'].replace({'no': 0, 'yes': 1})
    data['edjefe'] = data['edjefe'].astype('int')
    meaneduc_nan=data[data['meaneduc'].isnull()][['Id','idhogar','escolari']]
    me=meaneduc_nan.groupby('idhogar')['escolari'].mean().reset_index()
    for row in meaneduc_nan.iterrows():
        idx=row[0]
        idhogar=row[1]['idhogar']
        m=me[me['idhogar']==idhogar]['escolari'].tolist()[0]
        data.at[idx, 'meaneduc']=m
        data.at[idx, 'SQBmeaned']=m*m
        
    return data

In [ ]:
# train = data_cleaning(train)
# test = data_cleaning(test)
all_data = data_cleaning(all_data)

In [ ]:
# Tranform One-hot variables into Categorical variables
def onehot2cat(data, cat_col_new, cat_col_olds):
    cat_col = data[cat_col_olds].idxmax(1)
    cat_col.name = cat_col_new
    cat_col = cat_col.astype('category')
    data = pd.concat([data, cat_col], axis=1)
    data = data.drop(cat_col_olds, axis=1)
    return data

In [ ]:
cat_col_new_list = ['pared', 'piso', 'techo', 'abastagua', 'sanitario',
                'energcocinar', 'elimbasu', 'estadocivil',
                'parentesco', 'tipovivi', 'lugar', 'area']
cat_col_dict = {}
for cat_col_new in cat_col_new_list:
    cat_col_olds = [s for s in train.columns.tolist() if s.startswith(cat_col_new)]
    cat_col_dict[cat_col_new] = cat_col_olds
    
cat_col_dict['electricity'] = ['public', 'planpri', 'noelec', 'coopele']
cat_col_dict['sex'] = ['male', 'female']

In [ ]:
for cat_col_new, cat_col_olds in cat_col_dict.items():
    print(cat_col_olds)
#     train = onehot2cat(train, cat_col_new, cat_col_olds)
#     test = onehot2cat(test, cat_col_new, cat_col_olds)
    all_data = onehot2cat(all_data, cat_col_new, cat_col_olds)

In [ ]:
# Encode one-hot variables into numeric
# like (bad, regular, good) -> (0 ,1, 2)
def onehot2num(data, status_col_new, status_col_olds):
    status_df = data[status_col_olds]
    status_df.columns = list(range(len(status_col_olds)))
    num_col = status_df.idxmax(1)
    num_col.name = status_col_new
    data = pd.concat([data, num_col], axis=1)
    data = data.drop(status_col_olds, axis=1)
    return data

In [ ]:
status_col_new_list = ['epared', 'etecho', 'eviv', 'instlevel']
status_col_dict = {}
for status_col_new in status_col_new_list:
    status_col_olds = [s for s in train.columns.tolist() if s.startswith(status_col_new)]
    status_col_dict[status_col_new] = status_col_olds

In [ ]:
for status_col_new, status_col_olds in status_col_dict.items():
    print(status_col_olds)
#     train = onehot2num(train, status_col_new, status_col_olds)
#     test = onehot2num(test, status_col_new, status_col_olds)
    all_data = onehot2num(all_data, status_col_new, status_col_olds)

In [ ]:
# Delete needless columns
needless_cols = ['r4t3', 'tamhog', 'tamviv', 'hhsize', 'v18q', 'v14a', 'agesq',
                 'mobilephone']
SQB_cols = [s for s in train.columns.tolist() if 'SQB' in s]
needless_cols.extend(SQB_cols)
# train = train.drop(needless_cols, axis=1)
# test = test.drop(needless_cols, axis=1)
all_data = all_data.drop(needless_cols, axis=1)

In [ ]:
# Encode overcrowdig variables into crowding rate
all_data['hacdor_rate'] = all_data['hogar_total'] / all_data['bedrooms']
all_data['hacapo_rate'] = all_data['hogar_total'] / all_data['rooms']
all_data = all_data.drop(['hacdor', 'hacapo'], axis=1)

In [ ]:
# Encode dummy variables into category type
dummy_col_list = ['cielorazo', 'dis', 'computer', 'television']
for dummy_col in dummy_col_list:
#     train[dummy_col] = train[dummy_col].astype('category')
#     test[dummy_col] = test[dummy_col].astype('category')
    all_data[dummy_col] = all_data[dummy_col].astype('category')

In [ ]:
all_data.shape

## Select samples like head of household

In [ ]:
head_df_y = all_data['parentesco']=='parentesco1'

cols_relate_head =['edjefa', 'edjefe']
head_df_X = all_data.drop(['parentesco', 'Id', 'idhogar'] + cols_relate_head, axis=1)

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, roc_curve, auc, accuracy_score, make_scorer
import lightgbm as lgb

auc_scorer = make_scorer(auc, greater_is_better=True)

gbm_cv_param = {
    'num_leaves':[12]
#     ,'min_data_in_leaf':[9]
#     ,'max_depth':[14]
}
gbm_cv = GridSearchCV(
    lgb.LGBMClassifier(objective='binary', class_weight='balanced', seed=0)
    , gbm_cv_param
    , scoring=auc_scorer
    , cv=5
)
gbm_cv.fit(head_df_X, head_df_y)
gbm_cv.best_params_


# params = {'num_leaves': 13, 'min_data_in_leaf': 23, 'max_depth': 11, 'learning_rate': 0.09, 'feature_fraction': 0.74}
# gbm = lgb.LGBMClassifier(objective='multiclassova', class_weight='balanced', random_state=0)
# gbm.set_params(**params)
# gbm.fit(head_df_X, head_df_y)

In [ ]:
# params = {'num_leaves': 13, 'min_data_in_leaf': 23, 'max_depth': 11, 'learning_rate': 0.09, 'feature_fraction': 0.74}
gbm = lgb.LGBMClassifier(objective='binary', class_weight='balanced', random_state=0)
gbm.set_params(**(gbm_cv.best_params_))
gbm.fit(head_df_X, head_df_y)
y_pred_prob = gbm.predict_proba(head_df_X)[:,1]
y_pred = y_pred_prob > 0.5

In [ ]:
cm = confusion_matrix(head_df_y, y_pred)
cm_df_columns = ['pred_'+str(i) for i in range(cm.shape[0])]
cm_df_index = ['true_'+str(i) for i in range(cm.shape[0])]
cm_df = pd.DataFrame(data=cm, columns=cm_df_columns, index=cm_df_index)

ac = accuracy_score(head_df_y, y_pred)
fpr, tpr, thresholds = roc_curve(head_df_y, y_pred_prob)
auc_score = auc(fpr, tpr)
print("confusion matrix: \n", cm_df)
print("accuracy score: \n", ac)
print("auc score: \n", auc_score)

In [ ]:
ft_imp = pd.DataFrame({'features':head_df_X.columns,
                       'importance':gbm.feature_importances_})
ft_imp.sort_values(by='importance', ascending=False).iloc[0:10]

In [ ]:
pred_df = pd.DataFrame({'Id':all_data['Id'],
                            'true':head_df_y,
                            'pred':y_pred
                           })
headish_idx =  pred_df.query('true==0 & pred==1')['Id']

In [ ]:
headish_idx_train = []
for train_id in train['Id']:
    if train_id in headish_idx:
        headish_idx_train.append(train_id)

print('additional sample size:{}'.format(len(headish_idx_train)))

## Simple LightGBM

In [ ]:
train_headish = all_data.loc[train.index]
train_headish['Target'] = train_target
train_headish.head()
train_headish = train_headish.loc[headish_idx_train]
train_headish = train_headish.drop('parentesco', axis=1)

In [ ]:
train = all_data.loc[train.index]
train['Target'] = train_target
train = train.query('parentesco=="parentesco1"')
train = train.drop('parentesco', axis=1)

In [ ]:
test = all_data.loc[test.index]
test = test.drop('parentesco', axis=1)

In [ ]:
# Split data
train_Id = train['Id'] # individual ID
train_idhogar = train['idhogar'] # household ID
train_y = train['Target'] # Target value
train_X = train.drop(['Id', 'Target', 'idhogar'], axis=1) # features

train_headish_Id = train_headish['Id'] # individual ID
train_headish_idhogar = train_headish['idhogar'] # household ID
train_headish_y = train_headish['Target'] # Target value
train_headish_X = train_headish.drop(['Id', 'Target', 'idhogar'], axis=1) # features

test_Id = test['Id'] # individual ID
test_idhogar = test['idhogar'] # household ID
test_X = test.drop(['Id', 'idhogar'], axis=1) # features


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_X, train_y, test_size=0.1, random_state=0)

X_train = pd.concat([X_train, train_headish_X], axis=0)
y_train = pd.concat([y_train, train_headish_y], axis=0)

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, f1_score, make_scorer
import lightgbm as lgb



F1_scorer = make_scorer(f1_score, greater_is_better=True, average='macro')

# gbm_param = {
#     'num_leaves':[210]
#     ,'min_data_in_leaf':[9]
#     ,'max_depth':[14]
# }
# gbm = GridSearchCV(
#     lgb.LGBMClassifier(objective='multiclassova', class_weight='balanced', seed=0)
#     , gbm_param
#     , scoring=F1_scorer
# )


# params = {'num_leaves': 13, 'min_data_in_leaf': 23, 'max_depth': 11, 'learning_rate': 0.09, 'feature_fraction': 0.74}
gbm = lgb.LGBMClassifier(objective='multiclassova', class_weight='balanced', random_state=0)
# gbm.set_params(**params)

gbm.fit(X_train, y_train)
# gbm.best_params_

In [ ]:
import pickle
with open(os.path.join(working_dir, '20180822_lgbm_inflation.pickle'), mode='wb') as f:
    pickle.dump(gbm, f)

In [ ]:
y_test_pred = gbm.predict(X_test)
cm = confusion_matrix(y_test, y_test_pred)
f1 = f1_score(y_test, y_test_pred, average='macro')
print("confusion matrix: \n", cm)
print("macro F1 score: \n", f1)

In [ ]:
gbm = lgb.LGBMClassifier(objective='multiclassova', class_weight='balanced', random_state=0)
gbm.fit(train_X, train_y)

In [ ]:
pred = gbm.predict(test_X)
pred = pd.Series(data=pred, index=test_Id.values, name='Target')
pred = pd.concat([test_Id, pred], axis=1, join_axes=[test_Id.index])
pred.to_csv('20180822_lgbm_inflation.csv', index=False)